# Trabalho prático 2
### Processamento de Sinais Multimédia
####  Alunos: Diana Oliveira 52651; Beatriz Jesus 52749; Guilherme Cavalcanti 53077

In [ ]:
import numpy as np
from scipy import signal as ss
import scipy.io.wavfile as wav 
from IPython.display import Audio, display 
import matplotlib.pyplot as plt

##### Exercício 1

In [ ]:
# --------- Parâmetros Globais ---------
Fs = 48000  # Frequência de amostragem
temperatura = 20  # Temperatura em graus Celsius
recetorPos = (10,4) # Coordenadas do recetor (x,y) em metros
fonte1 = (10,6) # Coordenadas da fonte (x,y) em metros
fonte2 = (3,2) 
fonte3 = (17,8) 
fonte4 = (13,3) 
fonte5 = (6,9) 
fonte6 = (8,5) 
fonte7 = (12,7)

#S1 = (8,13) # reflexão da parede horizontal absorção=0.5
#S2 = (-7,7) # reflexão da parede vertical absorção=0.3
a = [0.5, 0.3]  # Coeficientes de absorção das paredes [parede vertical, parede horizontal]
c = 331.5 * np.sqrt(1 + temperatura/273) # Velocidade do som no ar em m/s 

# --------- MUTES ---------
#campo_direto_mute = False
mute1 = False


#Configurações Gerais
# --------- Converter Aúdio ---------
def audio_convertido(filename): # Adicionei Fs como argumento ou certifique-se que está definido globalmente
    fs_audio_org, audio_org = wav.read(filename) 
    audio_org = audio_org.astype(np.float32) # Converter para float32 
    max_val = np.max(np.abs(audio_org)) # Normalizar o áudio
    if max_val > 0: # Evitar divisão por zero
        audio_org /= max_val
    
    num_samples_new = int(len(audio_org) * Fs / fs_audio_org) # Calcular o número de amostras para o novo Fs
    audio_resampled = ss.resample(audio_org, num_samples_new) # Reamostrar o áudio para o novo Fs
    
    MAX_INT16 = 2**15 - 1 # Valor máximo para int16
    audio_resampled_scaled = audio_resampled * MAX_INT16 # Escalar para o intervalo de int16
    audio_resampled_clipped = np.clip(audio_resampled_scaled, -MAX_INT16, MAX_INT16) # Clipping para evitar overflow
    audio_final = audio_resampled_clipped.astype(np.int16) # Converter para int16
    
    return audio_final

# O aúdio utilizado já tem 16 bits mas fs=44100 Hz --> Tem que converter para Fs=48000 Hz
audio_resampled = audio_convertido("Borderline_UnmasteredWAV.wav")
wav.write("AudioConvertido.wav", Fs, audio_resampled)

# Os aúdios têm 44000hz e 24 bits --> converter para 48000hz e 16 bits
audio1_resampled = audio_convertido("instrumentos/01_Kick.wav")
wav.write("instrumentos/01_Kick_Convertido.wav", Fs, audio1_resampled)
audio2_resampled = audio_convertido("instrumentos/02_Snare.wav")
wav.write("instrumentos/02_Snare_Convertido.wav", Fs, audio2_resampled)
audio3_resampled = audio_convertido("instrumentos/03_Overheads.wav")
wav.write("instrumentos/03_Overheads_Convertido.wav", Fs, audio3_resampled)
audio4_resampled = audio_convertido("instrumentos/04_Percussion.wav")
wav.write("instrumentos/04_Percussion_Convertido.wav", Fs, audio4_resampled)
audio5_resampled = audio_convertido("instrumentos/05_Bass.wav")
wav.write("instrumentos/05_Bass_Convertido.wav", Fs, audio5_resampled)
audio6_resampled = audio_convertido("instrumentos/06_Gtr.wav")
wav.write("instrumentos/06_Gtr_Convertido.wav", Fs, audio6_resampled)
audio7_resampled = audio_convertido("instrumentos/07_LeadVox.wav")
wav.write("instrumentos/07_LeadVox_Convertido.wav", Fs, audio7_resampled)

# --------- Ler Aúdio Convertido  ---------
_ , audio = wav.read("AudioConvertido.wav")
fs_audio1, audio1 = wav.read("instrumentos/01_Kick_Convertido.wav")
fs_audio2, audio2 = wav.read("instrumentos/02_Snare_Convertido.wav")
fs_audio3, audio3 = wav.read("instrumentos/03_Overheads_Convertido.wav")
fs_audio4, audio4 = wav.read("instrumentos/04_Percussion_Convertido.wav")
fs_audio5, audio5 = wav.read("instrumentos/05_Bass_Convertido.wav")
fs_audio6, audio6 = wav.read("instrumentos/06_Gtr_Convertido.wav")
fs_audio7, audio7 = wav.read("instrumentos/07_LeadVox_Convertido.wav")


#calcular distancia das reflexoes
def distancia_reflexao(posFonte, parede): # pq as paredes sao os eixos do x e y
    if parede == 'horizontal':
        fonte_refletida = (posFonte[0], -posFonte[1])
    elif parede == 'vertical':
        fonte_refletida = (-posFonte[0], posFonte[1])

    distancia = np.sqrt((recetorPos[0] - fonte_refletida[0])**2 + (recetorPos[1] - fonte_refletida[1])**2)
    return distancia


# --------- Reverberação ---------
Comb_m = [int(0.03*Fs), int(0.05*Fs), int(0.07*Fs), int(0.09*Fs)]   # atraso do comb filters
AllPass_m = [int(0.005*Fs), int(0.0017*Fs)]  # atraso dos allpass filters de


def FBCF(alpha, Dc):
    b = np.zeros(Dc + 1); b[-1] = 1 
    a = np.zeros(Dc + 1); a[0] = 1; a[-1] = -alpha #ganho
    return b, a 

def AllPass(g, Da):
    b = np.zeros(Da + 1); b[0] = -g; b[-1] = 1 
    a = np.zeros(Da + 1); a[0] = 1; a[-1] = -g 
    return b, a


def getganho(RT60, m):
    ganho = 10**((-3 * m* (1/Fs)) / (RT60)) # 20*log10(g) / (m*T) = -60 / Tr  => log10(g) = -3*m*T / Tr , onde tr é rt60 e o T é o peirodo de amostragem (1/Fs)(passar dominio de amotras para tempo)
    return ganho


def reverb(campo, RT60):
    if RT60 == 0:
        return np.zeros_like(campo)
    sinal = np.zeros_like(campo)
    for m in Comb_m:
        ganho = getganho(RT60, m)
        bks, aks = FBCF(ganho, m)
        sinal += ss.lfilter(bks, aks, campo)

    for m in AllPass_m:
        bks, aks = AllPass(0.7, m)  
        sinal = ss.lfilter(bks, aks, sinal)
        
    return sinal

In [ ]:
# --------(d) Campo total = Campo direto + Primeiras reflexões + Campo Reverberante ---------
distancia_recetor_fonte = np.sqrt((np.abs(recetorPos[0])-np.abs(fonte1[0]))**2 + (np.abs(recetorPos[1])-np.abs(fonte1[1]))**2)  # Distância entre fonte e recetor
distancia_recetor_S1 = distancia_reflexao(fonte1, 'horizontal')  # Distância entre recetor e S1
distancia_recetor_S2 = distancia_reflexao(fonte1, 'vertical')  # Distância entre recetor e S2

# --------- Campo Direto ---------
t1 = distancia_recetor_fonte / c  # Tempo de atraso em segundos
m1 = int(t1 * Fs)  # Número de amostras de atraso
a1 = audio[:,0] * (1/distancia_recetor_fonte) # ganho (atenuação)

# --------- 1º Reflexão ---------
t2 = distancia_recetor_S1 / c  # Tempo de atraso em segundos
m2 = int(t2 * Fs)  # Número de amostras de atraso
a2 =  audio[:,0] * (1-a[0]) * (1/distancia_recetor_S1)  # ganho (coeficiente absorção * atenuação)

# --------- 2º Reflexão ---------
t3 = distancia_recetor_S2 / c  # Tempo de atraso em segundos
m3 = int(t3 * Fs)  # Número de amostras de atraso
a3 = audio[:,0] * (1-a[1]) * (1/distancia_recetor_S2) # ganho (coeficiente absorção * atenuação pela distância)


# Aplicar o atraso a cada um dos audios
atraso1 = np.zeros((m1))
a1 = np.concatenate((atraso1, a1), axis=0)# ganho do campo direto

atraso2 = np.zeros((m2))    
a2 = np.concatenate((atraso2, a2), axis=0) # ganho da 1ª reflexão

atraso3 = np.zeros((m3))
a3 = np.concatenate((atraso3, a3), axis=0)# ganho da 2ª reflexão

# Determinar o tamanho máximo
max_len = max(len(a1), len(a2), len(a3))

# Preencher zeros no final de cada array para igualar tamanhos
a1 = np.pad(a1, (0, max_len - len(a1)), mode='constant')
a2 = np.pad(a2, (0, max_len - len(a2)), mode='constant')
a3 = np.pad(a3, (0, max_len - len(a3)), mode='constant')


campo = a1 + a2 + a3 #campo direto, 1ª e 2ª reflexão
reflexoes = a2 + a3 #só as primeiras reflexões              



# --------- RT60 ---------

# Aplicar a reverberação para diferentes valores de RT60
campo0  = campo
campo05 = reverb(reflexoes, 0.5)
campo2  = reverb(reflexoes, 2)
campo10 = reverb(reflexoes, 10)
    

# Normalizar os sinais
campo_total_0 = campo0/np.max(np.abs(campo0))
campo_total_05 = campo05/np.max(np.abs(campo05))
campo_total_2 = campo2/np.max(np.abs(campo2))
campo_total_10 = campo10/np.max(np.abs(campo10))


# --------- Gravar e Reproduzir 10 segundos do Aúdio ---------
filename_exercício1 = ["Exercício1_RT60_0.wav", "Exercício1_RT60_0.5.wav", "Exercício1_RT60_2.wav", "Exercício1_RT60_10.wav"]
wav.write(filename_exercício1[0], Fs, campo_total_0.astype(np.int16))
wav.write(filename_exercício1[1], Fs,  campo_total_05.astype(np.int16))
wav.write(filename_exercício1[2], Fs,  campo_total_2.astype(np.int16))
wav.write(filename_exercício1[3], Fs,  campo_total_10.astype(np.int16))

print("Aúdio Original")
display(Audio("AudioConvertido.wav", rate=Fs))
print("RT60 = 0s")
display(Audio(campo_total_0[:10*Fs], rate=Fs))
print("RT60 = 0.5s")
display(Audio(campo_total_05[:10*Fs], rate=Fs))
print("RT60 = 2s")
display(Audio(campo_total_2[:10*Fs], rate=Fs))
print("RT60 = 10s")
display(Audio(campo_total_10[:10*Fs], rate=Fs))

In [ ]:
# --------- (a) Campo Direto ---------
t1 = distancia_recetor_fonte / c  # Tempo de atraso em segundos
m1 = int(t1 * Fs)  # Número de amostras de atraso
a1 = (1/distancia_recetor_fonte) # ganho (atenuação)
# ABKS E BKS
aks = [1]
bks = np.zeros(m1 + 1)
bks[m1] = a1
campo = ss.lfilter(bks, aks, audio[:,0]) # Teve que ser mono para não dar erro


display(Audio(campo[:10*Fs], rate=Fs))
#plt.plot(campo)
#plt.xlim(0, 3*Fs)
t = np.arange(len(campo)) / Fs 
plt.figure(figsize=(12,4))
plt.plot(t, campo, label='Campo Direto')
plt.xlim(0, 1.0)  # primeiros 2s, por exemplo
plt.legend()
plt.xlabel('Tempo [s]')
plt.ylabel('Amplitude')
plt.show()

In [ ]:
# --------- (b) 1ª Reflexão ---------
t2 = distancia_recetor_S1 / c  # Tempo de atraso em segundos
m2 = int(t2 * Fs)  # Número de amostras de atraso
a2 = (1-a[0]) * (1/distancia_recetor_S1)  # ganho (coeficiente absorção * atenuação)

# ABKS E BKS
aks = [1]
bks = np.zeros(m2 + 1)
bks[m2] = a2
campo = ss.lfilter(bks, aks, audio[:,0]) # Teve que ser mono para não dar erro

display(Audio(campo[:10*Fs], rate=Fs))
#plt.plot(campo)
#plt.xlim(0, 3*Fs)
t = np.arange(len(campo)) / Fs 
plt.figure(figsize=(12,4))
plt.plot(t, campo, label='Campo Direto')
plt.xlim(0, 1.0)  # primeiros 2s, por exemplo
plt.legend()
plt.xlabel('Tempo [s]')
plt.ylabel('Amplitude')
plt.show()

In [ ]:
# --------- 2ª Reflexão ---------
t3 = distancia_recetor_S2 / c  # Tempo de atraso em segundos
m3 = int(t3 * Fs)  # Número de amostras de atraso
a3 = (1-a[1]) * (1/distancia_recetor_S2)  # ganho (coeficiente absorção * atenuação)

# ABKS E BKS
aks = [1]
bks = np.zeros(m3 + 1)
bks[m3] = a3
campo = ss.lfilter(bks, aks, audio[:,0]) # Teve que ser mono para não dar erro


display(Audio(campo[:10*Fs], rate=Fs))
#plt.plot(campo)
#plt.xlim(0, 3*Fs)
t = np.arange(len(campo)) / Fs 
plt.figure(figsize=(12,4))
plt.plot(t, campo, label='Campo Direto')
plt.xlim(0, 1.0)  # primeiros 2s, por exemplo
plt.legend()
plt.xlabel('Tempo [s]')
plt.ylabel('Amplitude')
plt.show()

In [ ]:
# --------- (c) Campo Reverberante ---------

# Para se observar o RT60, aplica-se a um impulso
impulso = np.zeros(20 * Fs)
impulso[0] = 1

teste1 = reverb(impulso, 0.5)
teste2 = reverb(impulso, 2)
teste3 = reverb(impulso, 10)
    
t = np.arange(len(impulso)) / Fs 
plt.stem(t, teste1, label='RT60 = 0.5 s')
plt.xlim(0, 11)  # primeiros 11s, por exemplo
plt.legend()
plt.xlabel('Tempo [s]')
plt.ylabel('Amplitude')
plt.show()


plt.stem(t, teste2, label='RT60 = 2 s')
plt.xlim(0, 11)  # primeiros 11s, por exemplo
plt.legend()
plt.xlabel('Tempo [s]')
plt.ylabel('Amplitude')
plt.show()


plt.stem(t, teste3, label='RT60 = 10 s')
plt.xlim(0, 11)  # primeiros 11s, por exemplo
plt.legend()
plt.xlabel('Tempo [s]')
plt.ylabel('Amplitude')
plt.show()

##### Exercício 2

In [ ]:
# --------- Campo Binaural ---------
alfas = [0, 0.5, 0.8, 1]
RT60s = [0, 0.5, 2, 10]

audios = [
    audio1,  
    audio2,  
    audio3,  
    audio4, 
    audio5,  
    audio6,  
    audio7   
]

fontes = [
    fonte1,
    fonte2,
    fonte3,
    fonte4,
    fonte5,
    fonte6,
    fonte7
]


def calcular_angulos(fonte):
    fonte = np.asarray(fonte, dtype=float)
    recetor = np.asarray(recetorPos, dtype=float)

    # vetor da fonte para o recetor
    v = recetor - fonte
    dx, dy = v

    # ângulo em graus (intervalo [-180, 180])
    angulo = np.degrees(np.arctan2(dy, dx))

    # forçar múltiplos de 5 graus
    angulo = 5 * np.round(angulo / 5)

    return angulo


def filtros_em_fonte(fonte, alfa, RT60, posFonte):
    distancia_recetor_fonte = np.sqrt((np.abs(recetorPos[0])-np.abs(posFonte[0]))**2 + (np.abs(recetorPos[1])-np.abs(posFonte[1]))**2)  # Distância entre fonte e recetor
    distancia_recetor_S1 = distancia_reflexao(posFonte, 'horizontal')  # Distância entre recetor e S1
    distancia_recetor_S2 = distancia_reflexao(posFonte, 'vertical')  # Distância entre recetor e S2

    # --------- Campo Direto ---------
    t1 = distancia_recetor_fonte / c  # Tempo de atraso em segundos
    m1 = int(t1 * Fs)  # Número de amostras de atraso
    a1 = (1/distancia_recetor_fonte) # ganho (atenuação)

    # ABKS E BKS
    aks = [1]
    bks = np.zeros(m1 + 1)
    bks[m1] = a1
    campoDireto = ss.lfilter(bks, aks, fonte) # Teve que ser mono para não dar erro

    # --------- 1º Reflexão ---------
    t2 = distancia_recetor_S1 / c  # Tempo de atraso em segundos
    m2 = int(t2 * Fs)  # Número de amostras de atraso
    a2 = (1-alfa[0]) * (1/distancia_recetor_S1)  # ganho (coeficiente absorção * atenuação)

    # ABKS E BKS
    aks = [1]
    bks = np.zeros(m2 + 1)
    bks[m2] = a2
    campoReflexao1 = ss.lfilter(bks, aks, fonte) # Teve que ser mono para não dar erro

    # --------- 2ª Reflexão ---------
    t3 = distancia_recetor_S2 / c  # Tempo de atraso em segundos
    m3 = int(t3 * Fs)  # Número de amostras de atraso
    a3 = (1-alfa[1]) * (1/distancia_recetor_S2)  # ganho (coeficiente absorção * atenuação)

    # ABKS E BKS
    aks = [1]
    bks = np.zeros(m3 + 1)
    bks[m3] = a3
    campoReflexao2 = ss.lfilter(bks, aks, fonte) # Teve que ser mono para não dar erro

        
    campo = campoDireto + campoReflexao1 + campoReflexao2
    campoReverberado = reverb(campoReflexao1 + campoReflexao2, RT60)
    campoTotal = campo + campoReverberado
    return campoTotal


def aplicar_hrtf(campo, angulo):
    if np.abs(angulo) == 0:
        hrtfData = np.fromfile('compact/elev0/H0e000a.dat',dtype='short')
    elif np.abs(angulo) < 100:
        hrtfData = np.fromfile(f'compact/elev0/H0e0{int(np.abs(angulo))}a.dat',dtype='short')
    else:
        hrtfData = np.fromfile(f'compact/elev0/H0e{int(np.abs(angulo))}a.dat',dtype='short')

    leftimp00= hrtfData[0::2]
    rightimp00 = hrtfData[1::2]    

    if(angulo < 0):
        rightimp00, leftimp00 = leftimp00, rightimp00

    lchannel00=np.convolve(leftimp00,campo*1.0)
    rchannel00=np.convolve(rightimp00,campo*1.0)
    newData00 = np.zeros((len(lchannel00),2))
    newData00[:,0] = lchannel00/np.max(np.abs(lchannel00))*2**15
    newData00[:,1] = rchannel00/np.max(np.abs(rchannel00))*2**15
    return newData00
    

for alfa in alfas:
    soma = 0
    for i in range(len(audios)):
        fonte = audios[i]
        posFonte = fontes[i]
        angulo = calcular_angulos(posFonte)
        campoTotal = filtros_em_fonte(fonte, [alfa, alfa], 0.2 , posFonte)
        newData = aplicar_hrtf(campoTotal, angulo)
        soma += newData
    
    soma = soma / np.max(np.abs(soma)) * (2**15 - 1)
    wav.write(f"CampoBinaural_alfa_{alfa}.wav", Fs, soma.astype(np.int16))

In [ ]:
#Binaural com Reverberação

for rt60 in RT60s:
    soma = 0
    for i in range(len(audios)):
        fonte = audios[i]
        posFonte = fontes[i]
        angulo = calcular_angulos(posFonte)
        campoTotal = filtros_em_fonte(fonte, a, rt60, posFonte)
        newData = aplicar_hrtf(campoTotal, angulo)
        soma += newData

    soma = soma / np.max(np.abs(soma)) * (2**15 - 1)
    wav.write(f"CampoBinaural_RT60_{rt60}.wav", Fs, soma.astype(np.int16))

##### Exercício 3 

In [ ]:
# --------- (1) Codificação Ambisónica ---------
# Função para calcular o ângulo entre a fonte e o recetor
def angulo(posFonte, recetorPos):
    dx = posFonte[0] - recetorPos[0]
    dy = posFonte[1] - recetorPos[1]
    a = np.arctan2(dy, dx)  # radianos
    return a

# Função para codificação ambisónica de ordem 1 (sem Z porque trabalhamos em 2D)
def ambisonic_encode(S, angulo):
    W = S * (1/np.sqrt(2))
    X = S * np.cos(angulo)
    Y = S * np.sin(angulo)
    return W, X, Y

W = np.zeros_like(audio, dtype=np.float32)
X = np.zeros_like(audio, dtype=np.float32)
Y = np.zeros_like(audio, dtype=np.float32)

for i in range(len(fontes)):
    fonte = fontes[i]
    audio = audios[i]

    # Campo direto ambisónico
    angulo_direto = angulo(fonte, recetorPos)
    W1, X1, Y1 = ambisonic_encode(audio, angulo_direto)

    # 1ª reflexão ambisónica
    angulo_reflexao1 = angulo((-fonte[0], fonte[1]), recetorPos)  # reflexão na parede vertical
    W2, X2, Y2 = ambisonic_encode(audio, angulo_reflexao1)

    # 2ª reflexão ambisónica
    angulo_reflexao2 = angulo((fonte[0], -fonte[1]), recetorPos)  # reflexão na parede horizontal
    W3, X3, Y3 = ambisonic_encode(audio, angulo_reflexao2)

    # Soma dos componentes ambisónicos
    W_aux = W1 + W2 + W3
    X_aux = X1 + X2 + X3
    Y_aux = Y1 + Y2 + Y3

    W += W_aux
    X += X_aux
    Y += Y_aux

# Normalização dos sinais ambisónicos
max_val = max(np.max(np.abs(W)),np.max(np.abs(X)),np.max(np.abs(Y)))
W = W /max_val
X = X /max_val
Y = Y /max_val

ambisonic_WXY = np.column_stack((W, X, Y))
ambisonic_WXY = ambisonic_WXY / np.max(np.abs(ambisonic_WXY))
ambisonic_WXY = (ambisonic_WXY * (2**15-1)).astype(np.int16)
# Gravação dos sinais ambisónicos 
wav.write("Ambisonic.wav", Fs, ambisonic_WXY)

In [ ]:
# --------- (2) Definir geometria de localização dos altifalantes ---------
def gerar_altifalantes_circulares(cx, cy, raio):
    coordenadas = []
    for i in range(8):
        a = np.pi/2 + 2 * np.pi * i / 8  # Começa no π/2 (segundo a imagem do slide 41)
        x = cx + raio * np.cos(a)
        y = cy + raio * np.sin(a)
        coordenadas.append((round(x, 2), round(y, 2)))
    return coordenadas

centro = (10, 4)
raio = 2

altifalantes = gerar_altifalantes_circulares(centro[0], centro[1], raio)
#for i, coord in enumerate(altifalantes, start=1):
#    print(f"S{i} = {coord}")


In [ ]:
# --------- (3) Criação de um microfone ---------
p = 0.5 
microfone_pos = recetorPos  # Posição do microfone (x, y)
def microfone_ambisonico(angulo, p, W, X, Y):
    M = p * np.sqrt(2) * W  + (1 - p) * (np.cos(angulo) * X + np.sin(angulo) * Y)
    return M

direcoes = []
for alt in altifalantes:
    dx = alt[0] - microfone_pos[0]
    dy = alt[1] - microfone_pos[1]
    mag = np.sqrt(dx**2 + dy**2)
    dir_normalizada = (dx/mag, dy/mag)
    direcoes.append(dir_normalizada)

# Exemplo de visualização
for i, d in enumerate(direcoes, start=1):
    print(f"Direcao para S{i}: {d}, p={p}")